In [8]:
import pandas as pd
import numpy as np

project_name = "Aug_8"

import os
if not os.path.exists(f'{project_name}/control_results'):
    os.makedirs(f'{project_name}/control_results')

In [9]:
log = pd.read_csv(f'../log/{project_name}/koopman_results_log.csv')
log

,env_name,state_dim,u_dim,train_samples,val_samples,test_samples,Ksteps,train_steps,batch_size,initial_lr,...,normalize,best_val_Kloss,convergence_val_Kloss,test_Kloss,test_CovLoss,test_ControlLoss,model_path,num_params,encoder_num_params,m
0,DampingPendulum,2,1.0,60000,20000,20000,15,60000,64,0.001,...,False,1.290401e-02,1.291620e-02,1.291951e-02,1.172993e+07,1.413839e+03,../log/Aug_8/best_models/20250808_010058_model...,134168,134148,100
1,DampingPendulum,2,1.0,60000,20000,20000,15,60000,64,0.001,...,False,2.897012e-02,3.018802e-02,2.902783e-02,7.072261e-06,6.926630e+01,../log/Aug_8/best_models/20250808_010403_model...,134168,134148,100
2,DampingPendulum,2,1.0,60000,20000,20000,15,60000,64,0.001,...,False,7.533558e-02,7.556294e-02,7.556104e-02,6.422688e+00,2.126350e+01,../log/Aug_8/best_models/20250808_010719_model...,134168,134148,100
3,DampingPendulum,2,1.0,60000,20000,20000,15,60000,64,0.001,...,False,9.359109e-02,9.369736e-02,9.377694e-02,7.106797e-05,2.126347e+01,../log/Aug_8/best_models/20250808_011116_model...,134168,134148,100
4,DampingPendulum,2,1.0,60000,20000,20000,15,60000,64,0.001,...,False,1.111641e-02,1.113291e-02,1.111543e-02,2.938835e+04,2.972480e+03,../log/Aug_8/best_models/20250808_011517_model...,135218,135176,100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2458,Polynomial,3,NaN,16000,20000,20000,1,80000,64,0.001,...,False,8.558620e-05,8.826616e-05,8.949449e-05,1.035406e+03,NaN,../log/Aug_8/best_models/20250921_003726_model...,146697,145968,50
2459,Franka,17,7.0,16000,20000,20000,15,60000,64,0.001,...,False,4.379732e-06,4.406750e-06,4.384075e-06,3.256949e-04,1.956152e-08,../log/Aug_8/best_models/20250921_003510_model...,150932,149538,100
2460,Polynomial,3,NaN,60000,20000,20000,1,80000,64,0.001,...,False,5.002023e-05,5.111698e-05,3.741602e-05,9.555997e+02,NaN,../log/Aug_8/best_models/20250921_003945_model...,146697,145968,50
2461,Franka,17,7.0,4000,20000,20000,15,60000,64,0.001,...,False,2.471341e-08,2.585817e-08,2.493171e-08,7.892425e-02,1.944088e-03,../log/Aug_8/best_models/20250921_004023_model...,161234,158276,100


In [10]:
franka_results = pd.read_csv(f'../log/{project_name}/franka_control_results.csv')
franka_results

,env_name,seed,train_samples,encode_dim,use_control_loss,use_covariance_loss,max_eig,error_eight,error_star
0,Franka,17382,60000,17,False,False,1.000523,1.982986,2.544477
1,Franka,17382,60000,17,False,True,1.001274,3.384295,2.909771
2,Franka,17382,60000,17,True,False,1.000684,2.570720,2.180488
3,Franka,17382,60000,17,True,True,1.000466,1.981111,1.692435
4,Franka,17382,60000,34,False,False,1.001324,19.011537,17.723866
...,...,...,...,...,...,...,...,...,...
395,Franka,51194,4000,272,True,False,0.975590,31.877749,30.758160
396,Franka,51194,16000,272,True,False,0.940928,18.878293,32.632786
397,Franka,51194,1000,272,True,True,1.029612,1.817901,1.525587
398,Franka,51194,4000,272,True,True,0.991349,32.163100,14.040351


In [11]:
grouped = franka_results.groupby(['env_name', 'encode_dim', 'train_samples', 'use_control_loss', 'use_covariance_loss']).agg({
    'error_eight': ['mean', 'std'],
    'error_star': ['mean', 'std']
}).reset_index().sort_values(by=['train_samples', 'encode_dim'])
grouped.to_csv(f'{project_name}/control_results/Franka.csv', index=False)

In [12]:
grouped

env_name encode_dim train_samples use_control_loss use_covariance_loss  \
                                                                            
0    Franka         17          1000            False               False   
1    Franka         17          1000            False                True   
2    Franka         17          1000             True               False   
3    Franka         17          1000             True                True   
16   Franka         34          1000            False               False   
..      ...        ...           ...              ...                 ...   
63   Franka        136         60000             True                True   
76   Franka        272         60000            False               False   
77   Franka        272         60000            False                True   
78   Franka        272         60000             True               False   
79   Franka        272         60000             True                True   

   error_eight            error_star             
          mean        std       mean        std  
0     2.107360   1.052512   1.925200   0.922419  
1     1.880517   0.329411   1.815664   0.337805  
2     6.441222  10.658306   6.911326  12.097212  
3     1.839730   0.357825   1.666700   0.191181  
16    6.891838  12.271509   6.305510  10.774318  
..         ...        ...        ...        ...  
63   11.638516  17.235904  11.867956  14.203790  
76    2.426034   2.646393   2.016051   1.949919  
77    1.873710   0.750318   1.659628   0.469429  
78    1.836688   1.118414   1.554135   0.744937  
79    1.824505   0.424397   1.646200   0.279311  

[80 rows x 9 columns]

In [13]:
pendulum_results = pd.read_csv(f'../log/{project_name}/pendulum_control_results.csv')
pendulum_results

,env_name,encode_dim,use_control_loss,use_covariance_loss,train_samples,n_models,n_stable,stability_rate,finite_best_count,median_best_last10_L1,median_best_full_L1,median_best_control_cost,mean_best_last10_L1,mean_best_full_L1,mean_best_control_cost
0,DampingPendulum,2,False,False,1000,5,5,1.0,5,0.839624,1.658373,6838.069676,0.827942,1.952128,30855.017223
1,DampingPendulum,2,False,False,4000,5,5,1.0,5,0.745876,1.770350,5775.178679,0.608533,2.877944,41054.781140
2,DampingPendulum,2,False,False,16000,5,5,1.0,5,1.041948,1.209062,533.129206,0.749193,2.434110,31042.507158
3,DampingPendulum,2,False,False,60000,5,5,1.0,5,1.060279,1.212843,537.322711,0.746100,1.663338,19297.884580
4,DampingPendulum,2,False,True,1000,5,4,0.8,4,1.125503,1.497966,364.922519,1.093608,1.524554,550.836131
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,DoublePendulum,64,True,False,60000,5,2,0.4,2,1.906815,1.512023,8776.158235,1.906815,1.512023,8776.158235
156,DoublePendulum,64,True,True,1000,5,5,1.0,5,0.918572,1.009794,5168.580074,1.325567,1.461312,7925.013301
157,DoublePendulum,64,True,True,4000,5,5,1.0,5,0.872281,1.033193,20210.984095,0.915613,1.286920,39398.416365
158,DoublePendulum,64,True,True,16000,5,3,0.6,3,1.219856,1.183453,2321.832304,1.419414,1.556418,55228.674958


In [14]:
grouped = pendulum_results[['env_name', 'encode_dim', 'train_samples', 'use_control_loss', 'use_covariance_loss', 'stability_rate', 'median_best_last10_L1', 'median_best_control_cost']]

# change column name from median_best_last10_L1 to error
grouped = grouped.rename(columns={"median_best_last10_L1": "error", 'median_best_control_cost': 'control_cost'}).sort_values(by=['train_samples', 'encode_dim'])
grouped[grouped['env_name'] == 'DampingPendulum'].to_csv(f'{project_name}/control_results/DampingPendulum.csv', index=False)
grouped[grouped['env_name'] == 'DoublePendulum'].to_csv(f'{project_name}/control_results/DoublePendulum.csv', index=False)

In [15]:
grouped[grouped['env_name'] == 'DampingPendulum']

,env_name,encode_dim,train_samples,use_control_loss,use_covariance_loss,stability_rate,error,control_cost
0,DampingPendulum,2,1000,False,False,1.0,0.839624,6838.069676
4,DampingPendulum,2,1000,False,True,0.8,1.125503,364.922519
8,DampingPendulum,2,1000,True,False,1.0,1.151157,26350.914543
12,DampingPendulum,2,1000,True,True,1.0,0.777051,3464.542677
16,DampingPendulum,4,1000,False,False,1.0,1.036375,500.798782
...,...,...,...,...,...,...,...,...
63,DampingPendulum,16,60000,True,True,1.0,0.531944,22699.940799
67,DampingPendulum,32,60000,False,False,1.0,0.201702,1234.264073
71,DampingPendulum,32,60000,False,True,1.0,1.018207,519.011944
75,DampingPendulum,32,60000,True,False,1.0,0.014481,15530.342954


In [16]:
grouped[grouped['env_name'] == 'DoublePendulum']

,env_name,encode_dim,train_samples,use_control_loss,use_covariance_loss,stability_rate,error,control_cost
80,DoublePendulum,4,1000,False,False,1.0,0.493794,5981.731117
84,DoublePendulum,4,1000,False,True,1.0,0.311326,694.057666
88,DoublePendulum,4,1000,True,False,0.6,0.589538,4281.151189
92,DoublePendulum,4,1000,True,True,1.0,1.350587,1923.512407
96,DoublePendulum,8,1000,False,False,0.8,0.296663,1795.908015
...,...,...,...,...,...,...,...,...
143,DoublePendulum,32,60000,True,True,1.0,1.292997,1903.287333
147,DoublePendulum,64,60000,False,False,0.8,0.155232,846.488326
151,DoublePendulum,64,60000,False,True,1.0,0.674094,742.706018
155,DoublePendulum,64,60000,True,False,0.4,1.906815,8776.158235


In [17]:
isaacsim_results = pd.read_csv(f'../log/{project_name}/isaac_control_results.csv')
isaacsim_results = isaacsim_results.merge(log[['env_name', 'model_path', 'encode_dim', 'use_control_loss', 'use_covariance_loss', 'seed', 'train_samples']], on='model_path', how='left').drop(columns=['model_path', 'robot'])[['env_name','seed', 'train_samples', 'encode_dim', 'use_control_loss', 'use_covariance_loss', 'mean_JrPE', 'mean_survival_steps']].sort_values(by=['env_name', 'use_control_loss', 'use_covariance_loss'], ascending=[True, True, True])
isaacsim_results

,env_name,seed,train_samples,encode_dim,use_control_loss,use_covariance_loss,mean_JrPE,mean_survival_steps
0,G1,17382,200000,53,False,False,0.148349,53.688
4,G1,17382,200000,106,False,False,0.167348,55.219
8,G1,17382,200000,212,False,False,0.177962,58.296
12,G1,17382,200000,424,False,False,0.156307,58.436
16,G1,17382,200000,848,False,False,0.142367,60.102
...,...,...,...,...,...,...,...,...
786,Go2,51194,9333,280,True,True,0.668062,2.623
787,Go2,51194,37333,280,True,True,0.159144,96.116
797,Go2,51194,2333,560,True,True,0.619318,2.483
798,Go2,51194,9333,560,True,True,0.698119,2.742


In [18]:
grouped = isaacsim_results.groupby(['env_name', 'train_samples', 'encode_dim', 'use_control_loss', 'use_covariance_loss']).agg({
    'mean_JrPE': ['mean', 'std'],
    'mean_survival_steps': ['mean', 'std']
}).reset_index()
grouped[grouped['env_name'] == 'G1'].to_csv(f'{project_name}/control_results/G1.csv', index=False)
grouped[grouped['env_name'] == 'Go2'].to_csv(f'{project_name}/control_results/Go2.csv', index=False)

In [19]:
grouped

env_name train_samples encode_dim use_control_loss use_covariance_loss  \
                                                                             
0         G1          3333         53            False               False   
1         G1          3333         53            False                True   
2         G1          3333         53             True               False   
3         G1          3333         53             True                True   
4         G1          3333        106            False               False   
..       ...           ...        ...              ...                 ...   
155      Go2        140000        280             True                True   
156      Go2        140000        560            False               False   
157      Go2        140000        560            False                True   
158      Go2        140000        560             True               False   
159      Go2        140000        560             True                True   

    mean_JrPE           mean_survival_steps             
         mean       std                mean        std  
0    0.279228  0.047452              5.9154   0.812213  
1    0.279228  0.047452              5.9154   0.812213  
2    0.256019  0.026993             10.8048   1.749558  
3    0.256019  0.026993             10.8048   1.749558  
4    0.315275  0.031284              5.2890   0.722510  
..        ...       ...                 ...        ...  
155  0.126268  0.016136            101.1064  10.892510  
156  0.164964  0.009751             68.2394  10.873705  
157  0.162613  0.021591             64.2830  11.157816  
158  0.136721  0.016523             80.6608  16.914025  
159  0.124954  0.014971             95.9902   8.257797  

[160 rows x 9 columns]